##Trabalho Final da Disciplina d Análise Preditiva [IBMEC - 03/2023]
###MBA em BI & Data Science
###Karlena Holanda


# **Instalação do pacote mip**

In [1]:
!pip install mip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 47.1 MB/s eta 0:00:00


**Problema da Mochila**

Formulação do Modelo do problema da mochila (knapsack problem)

\begin{eqnarray}
\max \ 10\ x_{1} &+ 13\ x_{2}+18\ x_{3}+31\ x_{4}+7\ x_{5}+15\ x_{6}  \\
\mbox{sujeito a:}&  \\
11\ x_{1} + 15\ x_{2} &+20\ x_{3}+35\ x_{4}+10\ x_{5}+33\ x_{6} \leq 53  
\end{eqnarray}


Além disso $x$ é um vetor inteiro com valores não-negativos.

In [2]:
from mip import Model, maximize, xsum, BINARY, OptimizationStatus

In [3]:
coef_funcao_objetivo = [10, 13, 18, 31, 7, 15]
coef_restr = [11, 15, 20, 35, 10, 33]
termo_independente = 53 

In [4]:
I = range(len(coef_funcao_objetivo))
m = Model("knapsack-karlena")

In [5]:
x = [m.add_var(var_type=BINARY) for i in I]

In [6]:
m.objective = maximize(xsum(coef_funcao_objetivo[i] * x[i] for i in I))

In [7]:
funcao_objetivo=xsum(coef_funcao_objetivo[i] * x[i] for i in I)


In [8]:
print(funcao_objetivo)

+ 10var(0) + 13var(1) + 18var(2) + 31var(3) + 7var(4) + 15var(5) 


In [9]:
m += xsum(coef_restr[i] * x[i] for i in I) <= termo_independente
print(f'O modelo tem {m.num_cols} variável(eis), {m.num_rows} restrição(ões) e {m.num_nz} zero(s)')

O modelo tem 6 variável(eis), 1 restrição(ões) e 6 zero(s)


In [10]:
status = m.optimize(max_seconds=2)

In [11]:
status == OptimizationStatus.OPTIMAL

True

In [12]:
itens_selecionados = ["x"+str(i+1) for i in I if x[i].x >= 0.99]
print("Itens selecionados: {}".format(itens_selecionados))

Itens selecionados: ['x2', 'x4']


# **Instalação do pacote ortools**

In [13]:
!pip install ortools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 KB 26.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.22.1 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.22.1 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 4.22.1 which is incompatible.


**Exemplo de Programação Inteira**

In [14]:
from ortools.linear_solver import pywraplp

In [15]:
solver = pywraplp.Solver.CreateSolver('SCIP')

In [16]:
infinity = solver.infinity()
# x e y são variáveis inteiras não-negativas.
x = solver.IntVar(0.0, infinity, 'x')
y = solver.IntVar(0.0, infinity, 'y')

print('Número de variáveis =', solver.NumVariables())

Número de variáveis = 2


In [17]:
# x + 7 * y <= 17.5.
solver.Add(x + 7 * y <= 17.5)

# x <= 3.5.
solver.Add(x <= 3.5)

print('Número de restrições =', solver.NumConstraints())

Número de restrições = 2


In [18]:
# Maximize x + 10 * y.
solver.Maximize(x + 10 * y)

In [19]:
status = solver.Solve()

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
  print('Solução:')
  print('Valor objetivo =', solver.Objective().Value())
  print('x =', x.solution_value())
  print('y =', y.solution_value())
else:
  print('O problema não tem solução ótima.')



In [21]:
print('\nInformações sobre o processamento:')
print(f'Tempo de processamento em milisegundos foi de {solver.wall_time()}')
print(f'Quantidade de Iterações para resolver o problema foi de {solver.iterations()}')
print(f'Quantidade nós de branch-and-bound para resolver o problema foi de {solver.nodes()}')


Informações sobre o processamento:
Tempo de processamento em milisegundos foi de 18615
Quantidade de Iterações para resolver o problema foi de 0
Quantidade nós de branch-and-bound para resolver o problema foi de 1
